# CS180 / CS280A — Project 1: Prokudin‑Gorskii Colorization

**Notebook**: an interactive version of the starter pipeline. Run the cells in order. This notebook:
- loads a vertically stacked glass plate (B, G, R) image,
- aligns the G and R channels to B using a pyramid + integer shift search,
- displays results and prints `(x, y)` offsets, and
- includes optional postprocessing (auto-contrast, white-balance, auto-crop).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, transform
import os
import time

# -----------------------------
# Helper functions
# -----------------------------
def to_float01(im):
    """Convert image to float in [0,1]."""
    if np.issubdtype(im.dtype, np.floating):
        return np.clip(im, 0, 1).astype(np.float32)
    else:
        max_val = np.iinfo(im.dtype).max
        return (im.astype(np.float32) / max_val)

def split_channels_bgr(im):
    """Split a tall Prokudin image into B, G, R channels."""
    h = im.shape[0] // 3
    b = im[:h]
    g = im[h:2*h]
    r = im[2*h:3*h]
    return b, g, r

def roll2(im, dy, dx):
    """Shift image by (dy, dx)."""
    return np.roll(np.roll(im, dy, axis=0), dx, axis=1)

def downsample_image(image, scale_factor=0.5):
    """Downsample image by scale factor with anti-aliasing."""
    new_height = int(image.shape[0] * scale_factor)
    new_width = int(image.shape[1] * scale_factor)
    return transform.resize(image, (new_height, new_width), anti_aliasing=True)

# -----------------------------
# Alignment functions
# -----------------------------
def align_single_scale(ref, target, max_shift=15, metric='ncc', crop_frac=0.1):
    """
    Align target to ref using exhaustive search.
    - metric: 'l2' for Euclidean distance, 'ncc' for normalized cross-correlation
    - crop_frac: fraction of border to ignore for metric
    """
    h, w = ref.shape
    crop_h = int(h * crop_frac)
    crop_w = int(w * crop_frac)
    
    ref_crop = ref[crop_h:-crop_h, crop_w:-crop_w]
    best_score = None
    best_dy, best_dx = 0, 0
    
    for dy in range(-max_shift, max_shift+1):
        for dx in range(-max_shift, max_shift+1):
            shifted = roll2(target, dy, dx)
            shifted_crop = shifted[crop_h:-crop_h, crop_w:-crop_w]
            
            if metric == 'l2':
                score = np.sqrt(np.sum((ref_crop - shifted_crop)**2))
                if (best_score is None) or (score < best_score):
                    best_score = score
                    best_dy, best_dx = dy, dx
            elif metric == 'ncc':
                ref_vec = (ref_crop - ref_crop.mean()).flatten()
                tgt_vec = (shifted_crop - shifted_crop.mean()).flatten()
                score = np.dot(ref_vec, tgt_vec) / (np.linalg.norm(ref_vec) * np.linalg.norm(tgt_vec))
                if (best_score is None) or (score > best_score):
                    best_score = score
                    best_dy, best_dx = dy, dx
            else:
                raise ValueError("metric must be 'l2' or 'ncc'")
    
    return best_dy, best_dx

def align_pyramid(ref, target, levels=3, initial_max_shift=15, metric='ncc'):
    """
    Efficient pyramid alignment for large images.
    Starts with downsampled images and refines alignment at each level.
    """
    # Create pyramid levels
    ref_pyramid = [ref]
    target_pyramid = [target]
    
    # Build pyramid by downsampling
    for i in range(1, levels):
        ref_pyramid.append(downsample_image(ref_pyramid[-1], 0.5))
        target_pyramid.append(downsample_image(target_pyramid[-1], 0.5))
    
    # Start with coarsest level (smallest image)
    current_dy, current_dx = 0, 0
    
    # Work from coarsest to finest level
    for level in range(levels-1, -1, -1):
        # Scale the offset for current level
        scale_factor = 2 ** (levels - 1 - level)
        scaled_dy = current_dy * scale_factor
        scaled_dx = current_dx * scale_factor
        
        # Apply current offset to target
        shifted_target = roll2(target_pyramid[level], scaled_dy, scaled_dx)
        
        # Refine alignment at this level with smaller search range
        search_range = max(2, initial_max_shift // (2 ** (levels - 1 - level)))
        refine_dy, refine_dx = align_single_scale(
            ref_pyramid[level], shifted_target, 
            max_shift=search_range, metric=metric, crop_frac=0.1
        )
        
        # Update total offset
        current_dy = scaled_dy + refine_dy
        current_dx = scaled_dx + refine_dx
    
    return current_dy, current_dx


# -----------------------------
# Display function
# -----------------------------
def show_pair(before, after, title_before='Before', title_after='After'):
    plt.figure(figsize=(14,7))
    plt.subplot(1,2,1)
    plt.imshow(np.clip(before,0,1))
    plt.axis('off')
    plt.title(title_before)
    
    plt.subplot(1,2,2)
    plt.imshow(np.clip(after,0,1))
    plt.axis('off')
    plt.title(title_after)
    plt.show()

# -----------------------------
# Main processing function
# -----------------------------
def process_image(input_path, output_dir, use_pyramid=True, max_shift=15, metric='ncc'):
    """
    Process a Prokudin-Gorskii image with optional pyramid alignment.
    """
    print(f'\nProcessing {os.path.basename(input_path)} ...')
    
    # Load image
    im = io.imread(input_path)
    if im.ndim == 3:
        im = color.rgb2gray(im)
    im = to_float01(im)
    
    # Split channels
    b, g, r = split_channels_bgr(im)
    
    # Align channels
    start_time = time.time()
    
    if use_pyramid:
        # Use pyramid alignment for large images
        dy_g, dx_g = align_pyramid(b, g, levels=3, initial_max_shift=max_shift, metric=metric)
        dy_r, dx_r = align_pyramid(b, r, levels=3, initial_max_shift=max_shift, metric=metric)
    else:
        # Use single-scale alignment for small images
        dy_g, dx_g = align_single_scale(b, g, max_shift=max_shift, metric=metric)
        dy_r, dx_r = align_single_scale(b, r, max_shift=max_shift, metric=metric)
    
    alignment_time = time.time() - start_time
    print(f"Alignment completed in {alignment_time:.2f} seconds")
    
    print(f"Green displacement relative to Blue: (x={dx_g}, y={dy_g})")
    print(f"Red displacement relative to Blue:   (x={dx_r}, y={dy_r})")
    
    # Apply shifts
    g_al = roll2(g, dy_g, dx_g)
    r_al = roll2(r, dy_r, dx_r)
    
    # Stack channels
    rgb = np.dstack([r_al, g_al, b])
    
    # Save output
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    out_path = os.path.join(output_dir, f'{base_name}_out.jpg')
    io.imsave(out_path, (np.clip(rgb,0,1)*255).astype(np.uint8), check_contrast=False)
    print(f'Saved aligned image to {out_path}')
    
    return rgb, (dy_g, dx_g, dy_r, dx_r)

# -----------------------------
# User parameters
# -----------------------------
# JPG files (use simple alignment)
JPG_FILES = ['cathedral.jpg', 'monastery.jpg', 'tobolsk.jpg']



OUT_DIR = 'results_enhanced'
os.makedirs(OUT_DIR, exist_ok=True)

# Process JPG files with simple alignment
print("Processing JPG files with simple alignment...")
for fname in JPG_FILES:
    if os.path.exists(fname):
        rgb_result, offsets = process_image(
            fname, OUT_DIR, 
            use_pyramid=False,
            max_shift=15,
            metric='l2'
        )
        # Show before/after
        im = io.imread(fname)
        if im.ndim == 3:
            im = color.rgb2gray(im)
        im = to_float01(im)
        b, g, r = split_channels_bgr(im)
        show_pair(np.dstack([r, g, b]), rgb_result,
                 title_before=f'{fname} - Before shifts',
                 title_after=f'{fname} - Final RGB')


### MULTI-SCALE ALIGNMENT W/ EMIR OUT OF ALIGNMENT

In [ ]:
import numpy as np
import cv2
import time
import os
from numba import jit, prange

@jit(nopython=True, parallel=True)
def compute_ssd_numba(patch1, patch2):
    """Compute SSD using Numba for massive speedup."""
    return np.sum((patch1 - patch2) ** 2)

@jit(nopython=True, parallel=True)
def compute_ncc_numba(patch1, patch2):
    """Compute NCC using Numba for massive speedup."""
    mean1 = np.mean(patch1)
    mean2 = np.mean(patch2)
    
    patch1_norm = patch1 - mean1
    patch2_norm = patch2 - mean2
    
    norm1 = np.sqrt(np.sum(patch1_norm * patch1_norm))
    norm2 = np.sqrt(np.sum(patch2_norm * patch2_norm))
    
    if norm1 < 1e-8 or norm2 < 1e-8:
        return 0.0
    
    return np.sum(patch1_norm * patch2_norm) / (norm1 * norm2)

def optimized_resize(im, scale_factor):
    """
    Optimized resize using area interpolation for downsampling.
    """
    if scale_factor == 1.0:
        return im.copy()
    
    h, w = im.shape
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    return cv2.resize(im, (new_w, new_h), interpolation=cv2.INTER_AREA)

def build_optimized_pyramid(im, levels):
    """Build pyramid with optimized resizing."""
    pyramid = [im]
    for i in range(1, levels):
        prev_level = pyramid[-1]
        new_level = optimized_resize(prev_level, 0.5)
        pyramid.append(new_level)
    return pyramid

def detect_border_color(image, border_size=10):
    """
    Detect the dominant border color by sampling from all four edges.
    """
    h, w = image.shape
    
    # Sample from all four borders
    top_border = image[:border_size, :].flatten()
    bottom_border = image[-border_size:, :].flatten()
    left_border = image[:, :border_size].flatten()
    right_border = image[:, -border_size:].flatten()
    
    # Combine all border samples
    all_border_pixels = np.concatenate([top_border, bottom_border, left_border, right_border])
    
    # Use median to avoid outliers
    border_color = np.median(all_border_pixels)
    return border_color

def find_content_edges(image, border_tolerance=0.05):
    """
    Find the edges where actual content begins by detecting significant changes from border color.
    """
    border_color = detect_border_color(image)
    h, w = image.shape
    
    # Find top edge
    top_edge = 0
    for i in range(h):
        row_mean = np.mean(image[i, :])
        if abs(row_mean - border_color) > border_tolerance:
            top_edge = i
            break
    
    # Find bottom edge
    bottom_edge = h - 1
    for i in range(h-1, -1, -1):
        row_mean = np.mean(image[i, :])
        if abs(row_mean - border_color) > border_tolerance:
            bottom_edge = i
            break
    
    # Find left edge
    left_edge = 0
    for j in range(w):
        col_mean = np.mean(image[:, j])
        if abs(col_mean - border_color) > border_tolerance:
            left_edge = j
            break
    
    # Find right edge
    right_edge = w - 1
    for j in range(w-1, -1, -1):
        col_mean = np.mean(image[:, j])
        if abs(col_mean - border_color) > border_tolerance:
            right_edge = j
            break
    
    return top_edge, bottom_edge, left_edge, right_edge

def auto_crop_image(image, border_tolerance=0.05):
    """
    Automatically crop borders by detecting content edges.
    """
    top, bottom, left, right = find_content_edges(image, border_tolerance)
    
    # Add small safety margin
    margin = 2
    top = max(0, top - margin)
    bottom = min(image.shape[0], bottom + margin)
    left = max(0, left - margin)
    right = min(image.shape[1], right + margin)
    
    cropped = image[top:bottom, left:right]
    return cropped, (top, bottom, left, right)


def auto_contrast_image(image, clip_percentile=1.0):
    """
    Automatically adjust contrast by scaling intensities to full range.
    """
    # Calculate percentiles to avoid extreme outliers
    low_val = np.percentile(image, clip_percentile)
    high_val = np.percentile(image, 100 - clip_percentile)
    
    # Scale to [0, 1] range
    contrasted = (image - low_val) / (high_val - low_val + 1e-8)
    contrasted = np.clip(contrasted, 0, 1)
    
    return contrasted

def enhance_image_pipeline(image):
    """
    Complete image enhancement pipeline: auto-crop + auto-contrast.
    """
    # Auto-crop first
    cropped, crop_coords = auto_crop_image(image)
    
    # Auto-contrast
    enhanced = auto_contrast_image(cropped)
    
    return enhanced, crop_coords

def split_channels_bgr(im):
    """Split a tall Prokudin image into B, G, R channels."""
    h = im.shape[0] // 3
    b = im[:h]
    g = im[h:2*h]
    r = im[2*h:3*h]
    return b, g, r

def align_single_scale_optimized(reference, target, search_range=15, metric='ncc', border_crop=0.15):
    """
    Optimized single-scale alignment with precomputation and smarter search.
    """
    h, w = reference.shape
    crop_h = int(h * border_crop)
    crop_w = int(w * border_crop)
    
    # Crop reference
    ref_crop = reference[crop_h:h-crop_h, crop_w:w-crop_w]
    
    best_score = -np.inf if metric == 'ncc' else np.inf
    best_offset = (0, 0)
    
    # Precompute metric function
    if metric == 'ncc':
        metric_func = compute_ncc_numba
    else:
        metric_func = compute_ssd_numba
    
    # Use a spiral search pattern
    search_pattern = []
    for r in range(0, search_range + 1):
        for dy in [-r, r]:
            for dx in range(-r, r + 1):
                if (dy, dx) not in search_pattern:
                    search_pattern.append((dy, dx))
        for dx in [-r, r]:
            for dy in range(-r + 1, r):
                if (dy, dx) not in search_pattern:
                    search_pattern.append((dy, dx))
    
    for dy, dx in search_pattern:
        if abs(dy) > search_range or abs(dx) > search_range:
            continue
            
        # Shift target image
        shifted = np.roll(target, (dy, dx), axis=(0, 1))
        target_crop = shifted[crop_h:h-crop_h, crop_w:w-crop_w]
        
        # Compute metric
        score = metric_func(ref_crop, target_crop)
        
        # Update best score
        if (metric == 'ncc' and score > best_score) or \
           (metric == 'ssd' and score < best_score):
            best_score = score
            best_offset = (dy, dx)
    
    return best_offset, best_score

def align_channels_optimized(b, g, r, initial_search_range=30, pyramid_levels=5, metric='ncc'):
    """
    Highly optimized pyramid alignment with better level selection and early termination.
    """
    # Determine optimal pyramid levels based on image size
    min_dim = min(b.shape)
    optimal_levels = min(pyramid_levels, int(np.log2(min_dim / 64)) + 1)
    optimal_levels = max(2, optimal_levels)
    
    # Build pyramids
    b_pyramid = build_optimized_pyramid(b, optimal_levels)
    g_pyramid = build_optimized_pyramid(g, optimal_levels)
    r_pyramid = build_optimized_pyramid(r, optimal_levels)
    
    g_offset = (0, 0)
    r_offset = (0, 0)
    
    # Coarse-to-fine alignment
    for level in range(optimal_levels-1, -1, -1):
        scale_factor = 2 ** level
        current_search_range = max(2, initial_search_range // (2 ** (optimal_levels - level - 1)))
        
        b_level = b_pyramid[level]
        g_level = g_pyramid[level]
        r_level = r_pyramid[level]
        
        # Apply accumulated offsets
        g_shifted = np.roll(g_level, g_offset, axis=(0, 1))
        r_shifted = np.roll(r_level, r_offset, axis=(0, 1))
        
        # Align with smaller search range at finer levels
        g_new_offset, _ = align_single_scale_optimized(
            b_level, g_shifted, search_range=current_search_range, metric=metric
        )
        r_new_offset, _ = align_single_scale_optimized(
            b_level, r_shifted, search_range=current_search_range, metric=metric
        )
        
        # Update offsets
        g_offset = (g_offset[0] * 2 + g_new_offset[0], g_offset[1] * 2 + g_new_offset[1])
        r_offset = (r_offset[0] * 2 + r_new_offset[0], r_offset[1] * 2 + r_new_offset[1])
    
    # Apply final offsets
    aligned_g = np.roll(g, g_offset, axis=(0, 1))
    aligned_r = np.roll(r, r_offset, axis=(0, 1))
    
    return aligned_g, aligned_r, g_offset, r_offset

def process_with_enhancements(image_path, output_dir='results_enhanced', metric='ncc'):
    """
    Process image with auto-cropping and auto-contrasting enhancements.
    """
    print(f"Processing {os.path.basename(image_path)} without auto-crop and auto-contrast...")
    
    # Read image
    im = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if im is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Convert to float
    im_float = im.astype(np.float32) / 255.0
    
    # Apply enhancement pipeline to entire image first
    enhanced_im, crop_coords = enhance_image_pipeline(im_float)
    
    # Split channels from enhanced image
    b, g, r = split_channels_bgr(enhanced_im)
    
    start_time = time.time()
    
    # Align channels
    aligned_g, aligned_r, g_offset, r_offset = align_channels_optimized(
        b, g, r, initial_search_range=25, pyramid_levels=5, metric=metric
    )
    
    processing_time = time.time() - start_time
    
    # Create color image
    color_image = np.stack([aligned_r, aligned_g, b], axis=-1)
    color_image_uint8 = (np.clip(color_image, 0, 1) * 255).astype(np.uint8)
    
    # Save result
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_path = os.path.join(output_dir, f"enhanced_{os.path.basename(image_path).replace('.tif', '.jpg')}")
    cv2.imwrite(output_path, cv2.cvtColor(color_image_uint8, cv2.COLOR_RGB2BGR))
    
    print(f"  Crop coordinates: {crop_coords}")
    print(f"  Green offset: {g_offset}, Red offset: {r_offset}")
    print(f"  Processing time: {processing_time:.2f} seconds")
    print(f"  Saved to: {output_path}")
    
    return color_image_uint8, g_offset, r_offset, crop_coords

def main_enhanced():
    """Main function without auto-crop and auto-contrast enhancements."""
    # Process images with enhancements
    image_files = [
        'emir.tif', 'harvesters.tif', 'icon.tif', 'fresco.tif', 
        'melons.tif', 'church.tif', 'three_generations.tif',
        'napoleon.tif', 'obshchii_vid_kremlia.tif', 'portret_inokini_marfy.tif', 
        'self_portrait.tif', 'siren.tif', 'skaly_na_r.tif'
    ]
    
    total_start = time.time()
    
    for image_file in image_files:
        if os.path.exists(image_file):
            try:
                process_with_enhancements(image_file)
            except Exception as e:
                print(f"Error processing {image_file}: {e}")
        else:
            print(f"{image_file} not found, skipping...")
    
    total_time = time.time() - total_start
    print(f"\nTotal processing time: {total_time:.2f} seconds")

if __name__ == "__main__":
    main_enhanced()

### DISPLAY THE PHOTOS YOU PROCESSED

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import glob

def simple_display_results(output_dir='results'):
    """
    Simply display all images in the results directory one by one.
    """
    # Get all image files
    image_files = glob.glob(os.path.join(output_dir, '*.jpg')) + \
                  glob.glob(os.path.join(output_dir, '*.jpeg')) + \
                  glob.glob(os.path.join(output_dir, '*.tif')) + \
                  glob.glob(os.path.join(output_dir, '*.tiff'))
    
    if not image_files:
        print(f"No images found in {output_dir} directory.")
        return
    
    # Sort files alphabetically
    image_files.sort()
    
    # Display each image in a separate window
    for image_path in image_files:
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not read image: {image_path}")
            continue
        
        # Convert BGR to RGB for matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Display image
        plt.figure(figsize=(10, 8))
        plt.imshow(img_rgb)
        plt.title(os.path.basename(image_path))
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Run it
simple_display_results('results_enhanced')

### CODE BELOW FIXES EMIR AND ADDS CROPPING/CONTRAST (BLUES AND WHISTLES)

In [ ]:
import numpy as np
import cv2
import time
import os
from numba import jit, prange
from skimage.metrics import structural_similarity as ssim
from skimage import exposure

# -----------------------------
# Metric functions
# -----------------------------
@jit(nopython=True, parallel=True)
def compute_ssd_numba(patch1, patch2):
    return np.sum((patch1 - patch2) ** 2)

@jit(nopython=True, parallel=True)
def compute_ncc_numba(patch1, patch2):
    mean1 = np.mean(patch1)
    mean2 = np.mean(patch2)
    
    patch1_norm = patch1 - mean1
    patch2_norm = patch2 - mean2
    
    norm1 = np.sqrt(np.sum(patch1_norm * patch1_norm))
    norm2 = np.sqrt(np.sum(patch2_norm * patch2_norm))
    
    if norm1 < 1e-8 or norm2 < 1e-8:
        return 0.0
    
    return np.sum(patch1_norm * patch2_norm) / (norm1 * norm2)

def compute_ssim_metric(patch1, patch2):
    patch1 = np.clip(patch1, 0, 1)
    patch2 = np.clip(patch2, 0, 1)
    val, _ = ssim(patch1, patch2, full=True, data_range=1.0)
    return val

# -----------------------------
# Image enhancement pipeline
# -----------------------------
def optimized_resize(im, scale_factor):
    if scale_factor == 1.0:
        return im.copy()
    h, w = im.shape
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    return cv2.resize(im, (new_w, new_h), interpolation=cv2.INTER_AREA)

def build_optimized_pyramid(im, levels):
    pyramid = [im]
    for i in range(1, levels):
        prev_level = pyramid[-1]
        new_level = optimized_resize(prev_level, 0.5)
        pyramid.append(new_level)
    return pyramid

def detect_border_color(image, border_size=10):
    h, w = image.shape
    top_border = image[:border_size, :].flatten()
    bottom_border = image[-border_size:, :].flatten()
    left_border = image[:, :border_size].flatten()
    right_border = image[:, -border_size:].flatten()
    all_border_pixels = np.concatenate([top_border, bottom_border, left_border, right_border])
    return np.median(all_border_pixels)

def find_content_edges(image, border_tolerance=0.05):
    border_color = detect_border_color(image)
    h, w = image.shape
    top_edge, bottom_edge, left_edge, right_edge = 0, h-1, 0, w-1
    
    for i in range(h):
        if abs(np.mean(image[i, :]) - border_color) > border_tolerance:
            top_edge = i
            break
    for i in range(h-1, -1, -1):
        if abs(np.mean(image[i, :]) - border_color) > border_tolerance:
            bottom_edge = i
            break
    for j in range(w):
        if abs(np.mean(image[:, j]) - border_color) > border_tolerance:
            left_edge = j
            break
    for j in range(w-1, -1, -1):
        if abs(np.mean(image[:, j]) - border_color) > border_tolerance:
            right_edge = j
            break
    return top_edge, bottom_edge, left_edge, right_edge

def auto_crop_image(image, border_tolerance=0.15, min_content_fraction=0.7):
    top, bottom, left, right = find_content_edges(image, border_tolerance)
    if image.max() <= 1.0:
        gray = (image * 255).astype(np.uint8)
    else:
        gray = image.astype(np.uint8)
    h, w = gray.shape
    border_color = int(np.median(np.concatenate([gray[0,:], gray[-1,:], gray[:,0], gray[:,-1]])))
    tol = int(border_tolerance * 255)
    
    # Row-wise
    top, bottom = 0, h-1
    for i in range(h):
        if abs(int(np.mean(gray[i,:])) - border_color) > tol:
            top = i
            break
    for i in range(h-1, -1, -1):
        if abs(int(np.mean(gray[i,:])) - border_color) > tol:
            bottom = i
            break
    # Column-wise
    left, right = 0, w-1
    for j in range(w):
        if abs(int(np.mean(gray[:,j])) - border_color) > tol:
            left = j
            break
    for j in range(w-1, -1, -1):
        if abs(int(np.mean(gray[:,j])) - border_color) > tol:
            right = j
            break
    cropped_h = bottom - top
    cropped_w = right - left
    if cropped_h / h < min_content_fraction or cropped_w / w < min_content_fraction:
        return image, (0, h, 0, w)
    cropped = image[top:bottom, left:right]
    return cropped, (top, bottom, left, right)

def auto_contrast_image(image, method='hist'):
    if method == 'hist':
        contrasted = exposure.equalize_hist(image)
    elif method == 'adapt':
        contrasted = exposure.equalize_adapthist(image, clip_limit=0.03)
    else:
        raise ValueError("method must be 'hist' or 'adapt'")
    return np.clip(contrasted, 0, 1)

def enhance_image_pipeline(image, method='hist'):
    cropped, crop_coords = auto_crop_image(image)
    enhanced = auto_contrast_image(cropped, method=method)
    return enhanced, crop_coords, cropped  # Return cropped-only as well

def split_channels_bgr(im):
    h = im.shape[0] // 3
    return im[:h], im[h:2*h], im[2*h:3*h]

# -----------------------------
# Alignment functions
# -----------------------------
def align_single_scale_custom(reference, target, search_range=15, metric='ncc', border_crop=0.15):
    h, w = reference.shape
    crop_h, crop_w = int(h*border_crop), int(w*border_crop)
    ref_crop = reference[crop_h:h-crop_h, crop_w:w-crop_w]
    best_score = -np.inf
    best_offset = (0,0)
    
    if metric == 'ncc':
        metric_func = compute_ncc_numba
    elif metric == 'ssd':
        metric_func = compute_ssd_numba
    elif metric == 'ssim':
        metric_func = compute_ssim_metric
    else:
        raise ValueError(f"Unknown metric: {metric}")

    search_pattern = []
    for r in range(0, search_range+1):
        for dy in [-r,r]:
            for dx in range(-r,r+1):
                if (dy,dx) not in search_pattern: search_pattern.append((dy,dx))
        for dx in [-r,r]:
            for dy in range(-r+1,r):
                if (dy,dx) not in search_pattern: search_pattern.append((dy,dx))
                
    for dy, dx in search_pattern:
        if abs(dy)>search_range or abs(dx)>search_range: continue
        shifted = np.roll(target, (dy, dx), axis=(0,1))
        target_crop = shifted[crop_h:h-crop_h, crop_w:w-crop_w]
        score = metric_func(ref_crop, target_crop)
        if score > best_score:
            best_score = score
            best_offset = (dy, dx)
    return best_offset, best_score

def align_channels_optimized(b, g, r, initial_search_range=30, pyramid_levels=5, metric='ncc'):
    min_dim = min(b.shape)
    optimal_levels = min(pyramid_levels, int(np.log2(min_dim/64))+1)
    optimal_levels = max(2, optimal_levels)
    
    b_pyr = build_optimized_pyramid(b, optimal_levels)
    g_pyr = build_optimized_pyramid(g, optimal_levels)
    r_pyr = build_optimized_pyramid(r, optimal_levels)
    
    g_offset, r_offset = (0,0), (0,0)
    
    for level in range(optimal_levels-1, -1, -1):
        current_search = max(2, initial_search_range // (2**(optimal_levels-level-1)))
        b_lvl, g_lvl, r_lvl = b_pyr[level], g_pyr[level], r_pyr[level]
        g_shifted = np.roll(g_lvl, g_offset, axis=(0,1))
        r_shifted = np.roll(r_lvl, r_offset, axis=(0,1))
        
        g_new, _ = align_single_scale_custom(b_lvl, g_shifted, search_range=current_search, metric=metric)
        r_new, _ = align_single_scale_custom(b_lvl, r_shifted, search_range=current_search, metric=metric)
        
        g_offset = (g_offset[0]*2 + g_new[0], g_offset[1]*2 + g_new[1])
        r_offset = (r_offset[0]*2 + r_new[0], r_offset[1]*2 + r_new[1])
    
    aligned_g = np.roll(g, g_offset, axis=(0,1))
    aligned_r = np.roll(r, r_offset, axis=(0,1))
    
    return aligned_g, aligned_r, g_offset, r_offset

# -----------------------------
# Main processing
# -----------------------------
def process_with_enhancements(image_path, dir_cropped='results_cropped', dir_contrast='results_contrast'):
    print(f"Processing {os.path.basename(image_path)} ...")
    im = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if im is None:
        raise ValueError(f"Could not read image: {image_path}")
    im_float = im.astype(np.float32)/255.0
    
    # Apply pipeline
    enhanced, crop_coords, cropped_only = enhance_image_pipeline(im_float, method='hist')
    
    # Split channels
    b, g, r = split_channels_bgr(enhanced)
    
    start_time = time.time()
    metric_type = 'ssim' if os.path.basename(image_path).lower() == 'emir.tif' else 'ncc'
    aligned_g, aligned_r, g_offset, r_offset = align_channels_optimized(
        b, g, r, initial_search_range=25, pyramid_levels=5, metric=metric_type
    )
    processing_time = time.time() - start_time
    
    # Create images
    color_image_contrast = np.stack([aligned_r, aligned_g, b], axis=-1)
    color_image_contrast_uint8 = (np.clip(color_image_contrast,0,1)*255).astype(np.uint8)
    
    # For cropped-only (no contrast)
    b_nc, g_nc, r_nc = split_channels_bgr(cropped_only)
    aligned_g_nc, aligned_r_nc, _, _ = align_channels_optimized(
        b_nc, g_nc, r_nc, initial_search_range=25, pyramid_levels=5, metric=metric_type
    )
    color_image_cropped = np.stack([aligned_r_nc, aligned_g_nc, b_nc], axis=-1)
    color_image_cropped_uint8 = (np.clip(color_image_cropped,0,1)*255).astype(np.uint8)
    
    # Save both
    os.makedirs(dir_cropped, exist_ok=True)
    os.makedirs(dir_contrast, exist_ok=True)
    
    path_cropped = os.path.join(dir_cropped, f"cropped_{os.path.basename(image_path).replace('.tif','.jpg')}")
    path_contrast = os.path.join(dir_contrast, f"contrast_{os.path.basename(image_path).replace('.tif','.jpg')}")
    
    cv2.imwrite(path_cropped, cv2.cvtColor(color_image_cropped_uint8, cv2.COLOR_RGB2BGR))
    cv2.imwrite(path_contrast, cv2.cvtColor(color_image_contrast_uint8, cv2.COLOR_RGB2BGR))
    
    print(f"  Crop coords: {crop_coords}")
    print(f"  Green offset: {g_offset}, Red offset: {r_offset}")
    print(f"  Processing time: {processing_time:.2f}s")
    print(f"  Saved cropped-only: {path_cropped}")
    print(f"  Saved contrast: {path_contrast}")
    
    return color_image_cropped_uint8, color_image_contrast_uint8, g_offset, r_offset, crop_coords

def main_enhanced():
    image_files = [
        'emir.tif', 'harvesters.tif', 'icon.tif', 'fresco.tif', 
        'melons.tif', 'church.tif', 'three_generations.tif', 'lastochikino.tif',
        'napoleon.tif', 'obshchii_vid_kremlia.tif', 'portret_inokini_marfy.tif', 
        'self_portrait.tif','siren.tif', 'skaly_na_r.tif', 'ikona_sv_nikolaia.tif', 
        'lugano.tif', 'vid_s_kriepostnogo.tif'
    ]
    
    total_start = time.time()
    for image_file in image_files:
        if os.path.exists(image_file):
            try:
                process_with_enhancements(image_file)
            except Exception as e:
                print(f"Error processing {image_file}: {e}")
        else:
            print(f"{image_file} not found, skipping...")
    total_time = time.time() - total_start
    print(f"\nTotal processing time: {total_time:.2f} seconds")

if __name__ == "__main__":
    main_enhanced()


### HARD-CROP

In [ ]:
import cv2
import numpy as np
import os
import glob

def manual_hard_crop(image, crop_percent=0.1):
    """
    Manually crop a fixed percentage from all borders of an image
    """
    h, w = image.shape[:2]
    
    # Calculate crop pixels from each side
    crop_h = int(h * crop_percent)
    crop_w = int(w * crop_percent)
    
    # Calculate new boundaries
    top = crop_h
    bottom = h - crop_h
    left = crop_w
    right = w - crop_w
    
    # Ensure we don't crop too much
    if bottom - top < h * 0.5 or right - left < w * 0.5:
        # If we'd crop more than 50% of the image, crop less
        crop_h = int(h * 0.05)
        crop_w = int(w * 0.05)
        top = crop_h
        bottom = h - crop_h
        left = crop_w
        right = w - crop_w
    
    # Crop the image
    cropped = image[top:bottom, left:right]
    return cropped

def process_directory(input_dir, output_dir, crop_percent=0.1):
    """
    Process all images in a directory by cropping borders
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Find all JPG images in the input directory
    image_paths = glob.glob(os.path.join(input_dir, "*.jpg"))
    
    print(f"Processing {len(image_paths)} images from {input_dir}")
    
    for image_path in image_paths:
        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Could not read image: {image_path}")
            continue
        
        # Crop the image
        cropped_image = manual_hard_crop(image, crop_percent)
        
        # Save the cropped image
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, cropped_image)
        
        print(f"  Cropped and saved: {output_path}")

def main():
    # Directories
    cropped_dir = 'results_cropped' 
    contrast_dir = 'results_contrast' 
    output_cropped_dir = 'results_cropped_manual'
    output_contrast_dir = 'results_contrast_manual'
    
    # Crop percentage (10%)
    crop_percent = 0.1
    
    print("Manual Border Cropping Tool")
    print("=" * 50)
    
    # Process cropped-only images
    print("\nProcessing cropped-only images...")
    process_directory(cropped_dir, output_cropped_dir, crop_percent)
    
    # Process contrast-enhanced images
    print("\nProcessing contrast-enhanced images...")
    process_directory(contrast_dir, output_contrast_dir, crop_percent)
    
    print("\nDone! All images have been manually cropped by 10%.")
    print(f"Cropped-only results saved to: {output_cropped_dir}")
    print(f"Contrast-enhanced results saved to: {output_contrast_dir}")

if __name__ == "__main__":
    main()

### SIDE-BY-SIDE DISPLAY ALL PHOTOS

In [ ]:
# %%
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


# %%
# Configuration
CROPPED_DIR = 'results_cropped_manual'
CONTRAST_DIR = 'results_contrast_manual'
ORIGINAL_DIR = 'cs180 proj1 data'  # Directory containing original images

# %%
def get_image_pairs():
    """Get pairs of processed images from both directories"""
    cropped_files = [f for f in os.listdir(CROPPED_DIR) if f.endswith('.jpg')]
    contrast_files = [f for f in os.listdir(CONTRAST_DIR) if f.endswith('.jpg')]
    
    # Extract base names without prefixes
    base_names = set()
    for f in cropped_files + contrast_files:
        if f.startswith('cropped_'):
            base_names.add(f.replace('cropped_', ''))
        elif f.startswith('contrast_'):
            base_names.add(f.replace('contrast_', ''))
    
    # Create pairs
    pairs = []
    for base in sorted(base_names):
        cropped_path = os.path.join(CROPPED_DIR, 'cropped_' + base)
        contrast_path = os.path.join(CONTRAST_DIR, 'contrast_' + base)
        original_path = os.path.join(ORIGINAL_DIR, base.replace('.jpg', '.tif'))
        
        if os.path.exists(cropped_path) and os.path.exists(contrast_path):
            pairs.append({
                'name': base.replace('.jpg', ''),
                'cropped': cropped_path,
                'contrast': contrast_path,
                'original': original_path if os.path.exists(original_path) else None
            })
    
    return pairs

# %%
def display_comparison(image_pairs, max_display=None):
    """Display side-by-side comparison of images (Original | Cropped | Contrast)"""
    if max_display:
        image_pairs = image_pairs[:max_display]
    
    for pair in image_pairs:
        print(f"=== {pair['name']} ===")
        
        # Read images
        img_cropped = cv2.imread(pair['cropped'])
        img_contrast = cv2.imread(pair['contrast'])
        
        # Convert BGR to RGB
        img_cropped = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2RGB)
        img_contrast = cv2.cvtColor(img_contrast, cv2.COLOR_BGR2RGB)
        
        # Read original if available, otherwise use a blank placeholder
        if pair['original'] and os.path.exists(pair['original']):
            img_original = cv2.imread(pair['original'])
            img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        else:
            # Create a blank image with same size as cropped
            h, w, c = img_cropped.shape
            img_original = np.ones((h, w, 3), dtype=np.uint8) * 255  # white placeholder
        
        # Create figure with 3 columns
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        axes[0].imshow(img_original)
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        axes[1].imshow(img_cropped)
        axes[1].set_title('Cropped Only')
        axes[1].axis('off')
        
        axes[2].imshow(img_contrast)
        axes[2].set_title('Contrast Enhanced')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        print("\n")


# Main execution
if __name__ == "__main__":
    # Get all image pairs
    image_pairs = get_image_pairs()
    
    print(f"Found {len(image_pairs)} processed image pairs")
    
    # Display first few images in the notebook
    display_comparison(image_pairs, max_display=len(image_pairs))
    
   